In [8]:
import PyPDF4
import re
import pandas as pd
import openpyxl

# create an empty dataframe
df = pd.DataFrame()

# create a writer object for writing to an Excel file
writer = pd.ExcelWriter('all_sub_order.xlsx', engine='xlsxwriter')

# write the dataframe to the Excel file
df.to_excel(writer, sheet_name='Sheet1', index=False)

# save the Excel file
writer._save()

pdf_file = open('all_sub_order.pdf', 'rb')
pdf_reader = PyPDF4.PdfFileReader(pdf_file)

for page_num in range(1, pdf_reader.numPages):
    page = pdf_reader.getPage(page_num)
    pdf_text = page.extractText()
    # print(pdf_text)

    # Get invoice numberx
    match1 = re.search(r'Invoice\s*No.\s*(\S+)', pdf_text)
    if match1:
        invoice_number = match1.group(1)
        print("Invoice number: "+invoice_number)
    else:
        print("Invoice number not found in the text.")

    # Get invoice date
    match2 = re.search(r'Invoice\s*Date\s*(\S+)', pdf_text)
    if match2:
        invoice_date = match2.group(1).replace(".", "-")
        print("Invoice date: "+invoice_date)
    else:
        print("Invoice date not found in the text.")

    # Get Order date
    match3 = re.search(r'Order\s*Date\s*(\S+)', pdf_text)
    if match3:
        order_date = match3.group(1)
        print("Order date: "+order_date)
    else:
        print("Order date not found in the text.")

    match4 = re.search(r'Total\n(.*?)\nOther Charges', pdf_text, re.DOTALL)

    if match4:
        extracted_text = match4.group(1).strip()
        # print(extracted_text)
        lines = extracted_text.split("\n")
        lines1 = extracted_text.split('\n')[:len(lines)-9]
        # result = '\n'.join(lines1)
        description = ' '.join(filter(None, lines1))
        # print(result)

        hsn = int(lines[-9].replace("Rs.", ""))
        qty = int(lines[-8])
        gross_amount = float(lines[-7].replace("Rs.", ""))
        discount = float(lines[-6].replace("Rs.", ""))
        taxable_value = float(lines[-5].replace("Rs.", ""))

        taxes_string = lines[-4]
        match1 = re.search(r'@(.+?)%', taxes_string)
        if match1:
            taxes = float(match1.group(1))

        total = float(lines[-1].replace("Rs.", ""))

        print("Description = ", description)
        print("HSN = ", hsn)
        print("Qty = ", qty)
        print("Gross Amount = ", gross_amount)
        print("Discount = ", discount)
        print("Taxable Value = ", taxable_value)
        print("Taxes = ", str(taxes))
        print("Total = ", total)
        print("")

    print("")

    pattern = r"Order No\.\s+(.*?)\s+Fold Here"
    # print(pdf_text)
    match5 = re.search(pattern, pdf_text, re.DOTALL)

    if match5:
        result2 = match5.group(1)
        # print(result2.strip())
        lines2 = result2.split("\n")
        # print("SKU = "+lines2[0])
        sku = str(lines2[0])
        # print("Size = "+lines2[1])
        size = lines2[1]
        # print("Qty = "+lines2[2])
        qty = int(lines2[2])
        # print("Color = "+lines2[3])
        color = lines2[3]
        # print("Order No. = "+lines2[4])
        order_no = lines2[4]
    else:
        print("No match found.")

    print(match1 and match2 and match3 and match4 and match5)

    # Read the existing Excel file into a pandas dataframe
    df_existing = pd.read_excel('my_file.xlsx')

    if match1 and match2 and match3 and match4 and match5:
        data = {
            "INVOICE DATE": [invoice_date],
            "HSN": [hsn],
            "ORDERNO" : [order_no],
            "SHIPTO STATE" : ["Pending"],
            "SKU" : [sku],
            "SIZE" : [size],
            "QTY" : [qty],
            "COLOR" : [color],
            "DESCRIPTION": [description],
            "GROSS AMOUNT": [gross_amount],
            "DISCOUNT": [discount],
            "TAXABLE VALUE": [taxable_value],
            "TAXES": [taxes],
            "TOTAL": [total ]
        }

        # read the existing excel file into a dataframe
        df = pd.read_excel('my_file.xlsx')

        # append the new data to the dataframe
        df = df._append(pd.DataFrame(data), ignore_index=True)

        # write the updated dataframe back to the same excel file
        df.to_excel('my_file.xlsx', index=False, header=True)
       
    print("\n\n\n")
pdf_file.close()

# Open the workbook
workbook = openpyxl.load_workbook('my_file.xlsx')

# Select the worksheet you want to modify
worksheet = workbook['Sheet1']

# Iterate over the columns and set their width based on the maximum length of cell content
for column_cells in worksheet.columns:
    max_length = 0
    column = column_cells[0].column_letter  # Get the column letter
    for cell in column_cells:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2)  # Adjust the width a bit to avoid truncating
    worksheet.column_dimensions[column].width = adjusted_width
    

# Save the workbook
workbook.save('all_sub_order.xlsx')


Invoice number: rv23a235578
Invoice date: 20-03-2023
Order date: 18.03.2023
Description =  Shivansh Creation Women   Reyon Printed Maternity   Kurta - M  
HSN =  6204
Qty =  1
Gross Amount =  404.0
Discount =  0.0
Taxable Value =  384.76
Taxes =  5.0
Total =  404.0


<re.Match object; span=(422, 479), match='Order No.\n1443290244\nM\n1\nMustard\n32118969029>




Invoice number: rv23a235583
Invoice date: 20-03-2023
Order date: 19.03.2023
Description =  Shivansh Creation Women's   Crepe Printed Unstitiched   Dress Material  Suits  - Un   Stitched  
HSN =  540710
Qty =  1
Gross Amount =  334.0
Discount =  0.0
Taxable Value =  318.1
Taxes =  5.0
Total =  334.0


<re.Match object; span=(437, 499), match='Order No.\n1838955934\nUn Stitched\n1\nNA\n370533>




Invoice number: rv23a235579
Invoice date: 20-03-2023
Order date: 19.03.2023
Description =  Shivansh Creation Women's   Crepe Printed Unstitiched   Dress Material  Suits  - Un   Stitched  
HSN =  540710
Qty =  1
Gross Amount =  334.0
Dis